In [1]:
source ../modules/setup.tcl
set year 2017
set day 7

aoc::get-puzzle $year $day 1
aoc::get-puzzle $year $day 2
set input [string trim [aoc::get-input $year $day]]
jupyter::html "<h2>Input</h2>"
jupyter::display "text/plain" [string range $input 0 100]...;

(cached)

--- Day 7: Recursive Circus --- Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower. One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own. You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name , their weight , and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information. For example, if your list is the following: pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
 ...then you would be able to recreate the structure of the towers that looks like this: gyxo
 / 
 ugml - ebii
 / \ 
 | jptl
 | 
 | pbga
 / /
tknk --- padx - havc
 \ \
 | qoyq
 | 
 | ktlj
 \ / 
 fwft - cntj
 \ 
 xhth
 In this example, tknk is at the bottom of the tower (the bottom program ), and is holding up ugml , padx , and fwft . Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.) Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

(cached)

--- Part Two --- The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight , and until it's fixed, they're stuck here. For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower. In the example above, this means that for ugml 's disc to be balanced, gyxo , ebii , and jptl must all have the same weight, and they do: 61 . However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same: 
 
 ugml + ( gyxo + ebii + jptl ) = 68 + (61 + 61 + 61) = 251 
 padx + ( pbga + havc + qoyq ) = 45 + (66 + 66 + 66) = 243 
 fwft + ( ktlj + cntj + xhth ) = 72 + (57 + 57 + 57) = 243 
 As you can see, tknk 's disc is unbalanced: ugml 's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60 . Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

(cached)

Input

rmhcw (26)
dmmriu (61)
zjyaaoc (89) -> khekb, lywwhf
kjgaib (385)
nvvdii (28)
zsgegr (15)
azrafw (72)...

In [24]:
package require struct::tree
catch {programs destroy}
struct::tree programs;

In [25]:
proc ensure {tree node} {
    if {![$tree exists $node]} {      
        $tree rename [$tree insert root end] $node
    }
    return $node
}

proc unbalanced {} {
    set nodes [programs leaves]
    while {$nodes ne "root"} {
        set nn {}
        foreach l $nodes {
           set parent [programs parent $l]
           lappend nn $parent
           set ws {}
           foreach c [programs children $parent] {
               set tw 0
               programs walk $c x {incr tw [programs set $x w]}
               lappend ws $tw
           }
           if {[llength [lsort -unique $ws]] != 1} {
              return [programs children $parent]  
           }
        }
        set nodes $nn
    }
}

   

In [30]:
proc parts input {
    foreach l [split $input \n] {
        if {[regexp {([a-z]+) \(([0-9]+)\) -> (.*)} $l -> parent weight children]}  {
            programs set [ensure programs $parent] w $weight
            foreach child [string map {, {}} $children] {
                programs move $parent end [ensure programs $child]
            }
        } else {
            regexp {([a-z]+) \(([0-9]+)\)} $l -> parent weight
            programs set [ensure programs $parent] w $weight
        }
    }
    set result1 [programs children root]

    set ws {}
    foreach c [unbalanced] {
        set tw 0
        programs walk $c x {incr tw [programs set $x w]}
        lappend ws $c $tw
    }
    foreach {n w}  $ws {
        lappend dw($w) $n 
    }
    foreach n [array names dw] {
        if {[llength $dw($n)] > 1} {
            set correct $n
        } else {
            set incorrect $n
            set incorrectnode $dw($n)
        }
    }
    set diff [- $incorrect $correct]
    set result2 [- [programs get $incorrectnode w] $diff]
      puts [llength [programs leaves]]

    return [list $result1 $result2]
  }
aoc::results

930
Day1	ykpsek
Day2	1060
134298 microseconds per iteration


In [34]:
set tree {}

foreach l [split $input \n] {
    if {[regexp {([a-z]+) \(([0-9]+)\) -> (.*)} $l -> parent weight children]}  {
        set children [string map {, {}} $children]
        dict set tree $parent w $weight
        dict set tree $parent c $children
        foreach child [string map {, {}} $children] {
            dict set tree $child p $parent
        }
    } else {
        regexp {([a-z]+) \(([0-9]+)\)} $l -> parent weight
        dict set tree $parent w $weight
    }
}

proc empty? {v} {
    expr {$v eq {}}
}

proc root {tree} {
    lindex [dict filter $tree script {k v} {empty? [dict getdef $v p {}]}] 0
}

proc leaves {tree} {
    lsort -unique [dict map {x v}  [dict filter $tree script {k v} {empty? [dict getdef $v c {}]}] { lindex $x 0}]
}

puts "Part1: [root $tree]"
llength [leaves $tree]

Part1: ykpsek


930

In [37]:
proc weight {tree node} {
    incr w [dict get $tree $node w]
    foreach c [dict getdef $tree $node c {}] {
        incr w [weight $tree $c]
    }
    return $w
}

proc children {tree node} {
    dict getdef $tree $node c {}
}

In [43]:
lmap c [children $tree cumah] {list $c [weight $tree $c]}

{pqmyjtl 278} {wczux 278} {rfjilj 278}

In [44]:
dict get $tree cumah w

1069

In [54]:
proc weights {tree nodes} {
    foreach n $nodes {
        dict lappend ws [weight $tree $n] $n
    }
    return $ws
}

In [55]:
weights $tree [children $tree [root $tree]]

94250 {xgudb fucsb xjllex splbrdu} 94259 rsalq

In [56]:
weights $tree [children $tree rsalq]

9286 {sdxbol irqwcab jiaiwto viydtj} 9295 uduyfo

In [57]:
weights $tree [children $tree uduyfo]

1894 {amhaz zqfvypo dribos} 1903 cumah

In [58]:
weights $tree [children $tree cumah]

278 {pqmyjtl wczux rfjilj}

In [59]:
weight $tree cumah

1903

In [61]:
dict get $tree cumah w

1069

In [62]:
- 1069 9

1060